This notebook will be collected automatically at **6pm on Monday** from `/home/data_scientist/assignments/Week13` directory on the course JupyterHub server. If you work on this assignment on the course Jupyterhub server, just make sure that you save your work and instructors will pull your notebooks automatically after the deadline. If you work on this assignment locally, the only way to submit assignments is via Jupyterhub, and you have to place the notebook file in the correct directory with the correct file name before the deadline.

1. Make sure everything runs as expected. First, restart the kernel (in the menubar, select `Kernel` → `Restart`) and then run all cells (in the menubar, select `Cell` → `Run All`).
2. Make sure you fill in any place that says `YOUR CODE HERE`. Do not write your answer in anywhere else other than where it says `YOUR CODE HERE`. Anything you write anywhere else will be removed by the autograder.
3. Do not change the file path or the file name of this notebook.
4. Make sure that you save your work (in the menubar, select `File` → `Save and CheckPoint`)

## Problem 13.1. MongoDB

In this problem, we work with MongoDB from a Python program by using the pymongo database driver.

In [1]:
import os
import datetime
import json
import bson
import pymongo as pm

from nose.tools import assert_equal, assert_true, assert_is_instance

We use the historical weather data from [Weather Underground](http://www.wunderground.com/) to create a database. For more information on this data set, see [Week 4 Problem 1](https://github.com/UI-DataScience/info490-sp16/blob/master/Week4/assignments/w4p1.ipynb).

In [2]:
fpath = '/home/data_scientist/data/weather'
fname = 'weather_kord_2001_0101.json'

with open(os.path.join(fpath, fname)) as f:
    weather_json = json.load(f)

assert_is_instance(weather_json, dict)
assert_equal(set(weather_json.keys()), set(['current_observation', 'response', 'history']))
assert_true('observations' in weather_json['history'])

In [3]:
observations = weather_json['history']['observations']
print('There are {} dictionaries in the list.'.format(len(observations)))
print('The first element is\n{}'.format(observations[0]))

assert_is_instance(observations, list)
assert_true(all(isinstance(o, dict) for o in observations))

There are 24 dictionaries in the list.
The first element is
{'rain': '0', 'wdird': '360', 'icon': 'cloudy', 'vism': '14.5', 'tempm': '-10.6', 'dewptm': '-11.7', 'wdire': 'North', 'wspdm': '7.4', 'windchilli': '5.2', 'snow': '0', 'hum': '92', 'fog': '0', 'tornado': '0', 'precipi': '-9999.00', 'tempi': '12.9', 'visi': '9.0', 'wgusti': '-9999.0', 'date': {'hour': '00', 'mon': '01', 'min': '56', 'pretty': '12:56 AM CST on January 01, 2001', 'year': '2001', 'tzname': 'America/Chicago', 'mday': '01'}, 'hail': '0', 'pressurei': '30.38', 'precipm': '-9999.00', 'wspdi': '4.6', 'metar': 'METAR KORD 010656Z 36004KT 9SM BKN055 OVC095 M11/M12 A3034 RMK AO2 SLP285 T11061117 $', 'conds': 'Overcast', 'heatindexi': '-9999', 'wgustm': '-9999.0', 'utcdate': {'hour': '06', 'mon': '01', 'min': '56', 'pretty': '6:56 AM GMT on January 01, 2001', 'year': '2001', 'tzname': 'UTC', 'mday': '01'}, 'dewpti': '10.9', 'windchillm': '-14.9', 'heatindexm': '-9999', 'thunder': '0', 'pressurem': '1028.5'}


We connect to the course MongoDB cloud computing system, hosted by Microsoft Azure.

In [4]:
client = pm.MongoClient("mongodb://104.214.79.74:27017")

Since we are using a shared resource without authentication, we use your netid to create a database for each student.

In [5]:
# Filename containing user's netid
fname = '/home/data_scientist/users.txt'
with open(fname, 'r') as fin:
    netid = fin.readline().rstrip()

# We will delete our working directory if it exists before recreating.
dbname = 'assignment-{0}'.format(netid)

if dbname in client.database_names():
    client.drop_database(dbname)

print('Existing databases:', client.database_names())

assert_true(dbname not in client.database_names())

Existing databases: ['assignment-guo32', 'assignment-holgado2', 'assignment-jknguye2', 'test', 'test-thursto2', 'config']


## Inserting Data

- Create a new collection using the name `collection_name` and add new documents `data` to our MongoDB collection
- Return a list of object IDs as a validation of the insertion process.

In [6]:
def insert_data(db, collection_name, data):
    '''
    Creates a new collection using the name "collection_name" 
    and adds new documents `data` to our MongoDB collection.
    
    Parameters
    ----------
    data: A list of dictionaries.
    db: A pymongo.database.Database instance.
    collection_name: Name of new MongoDB collection.
    
    Returns
    -------
    A list of bson.ObjectId
    '''
    
    # YOUR CODE HERE
    temp = db[collection_name]
    inserted_ids = list(map(lambda item: temp.insert_one(item).inserted_id, data)) # for each item, insert it into the db, and
                                                                                    # get the corrsponding inserted_id
    
    return inserted_ids

In [7]:
inserted_ids = insert_data(client[dbname], '0101', observations)

print("New weather ID: ", inserted_ids)
print('Existing databases:', client.database_names())
print('Existing collections:', client[dbname].collection_names())

New weather ID:  [ObjectId('57155bfbdaf18100395cdb5b'), ObjectId('57155bfbdaf18100395cdb5c'), ObjectId('57155bfbdaf18100395cdb5d'), ObjectId('57155bfbdaf18100395cdb5e'), ObjectId('57155bfbdaf18100395cdb5f'), ObjectId('57155bfbdaf18100395cdb60'), ObjectId('57155bfcdaf18100395cdb61'), ObjectId('57155bfcdaf18100395cdb62'), ObjectId('57155bfcdaf18100395cdb63'), ObjectId('57155bfcdaf18100395cdb64'), ObjectId('57155bfcdaf18100395cdb65'), ObjectId('57155bfcdaf18100395cdb66'), ObjectId('57155bfcdaf18100395cdb67'), ObjectId('57155bfcdaf18100395cdb68'), ObjectId('57155bfcdaf18100395cdb69'), ObjectId('57155bfcdaf18100395cdb6a'), ObjectId('57155bfcdaf18100395cdb6b'), ObjectId('57155bfcdaf18100395cdb6c'), ObjectId('57155bfcdaf18100395cdb6d'), ObjectId('57155bfcdaf18100395cdb6e'), ObjectId('57155bfcdaf18100395cdb6f'), ObjectId('57155bfcdaf18100395cdb70'), ObjectId('57155bfddaf18100395cdb71'), ObjectId('57155bfddaf18100395cdb72')]
Existing databases: ['assignment-guo32', 'assignment-holgado2', 'assig

In [8]:
assert_is_instance(inserted_ids, list)
assert_true(all(isinstance(i, bson.objectid.ObjectId) for i in inserted_ids))

assert_true(dbname in client.database_names())
assert_true('0101' in client[dbname].collection_names())
assert_equal(client[dbname]['0101'].count(), len(observations))

## Retrieving Data

- Find all documents that have a given weather `condition` (e.g., `conds == "Clear"` or `conds == "Partly Cloudy"`)
- Return the `_id` values of all documents that match the search query.

In [9]:
def retrieve_data(collection, condition):
    '''
    Finds all documents that have a given weather `condition`
    and return the `_id` values of all documents that match the search query.
    
    Parameters
    ----------
    collection: A pymongo.Collection instance.
    condition: A string, e.g., "Clear", "Partly Cloudy", "Overcast".
    
    Returns
    -------
    A list of bson.ObjectId
    '''
    
    # YOUR CODE HERE
    result = [item['_id'] for item in collection.find({"conds": condition})]
    
    return result

In [10]:
clear_ids = retrieve_data(client[dbname]['0101'], 'Clear')
print(clear_ids)

[ObjectId('57155bfbdaf18100395cdb5e'), ObjectId('57155bfcdaf18100395cdb68'), ObjectId('57155bfcdaf18100395cdb69'), ObjectId('57155bfcdaf18100395cdb70'), ObjectId('57155bfddaf18100395cdb71'), ObjectId('57155bfddaf18100395cdb72')]


In [11]:
assert_is_instance(clear_ids, list)
assert_true(all(isinstance(i, bson.objectid.ObjectId) for i in clear_ids))

conds = {obs['conds'] for obs in observations}
for cond in conds:
    r = retrieve_data(client[dbname]['0101'], cond)
    n = [obs['_id'] for obs in observations if obs['conds'] == cond]
    assert_equal(len(r), len(n))
    assert_equal(set(r), set(n))

## Modifying Data

- Find all documents whose `conds` value is `"Clear"` and change the `conds` attribute to `Cloudy`.
- Return the number of documents modified as a validation of the process.

In [14]:
def modify_data(collection):
    '''
    Finds all documents whose "conds" value is "Clear"
    and change the "conds" attribute to "Cloudy".

    Parameters
    ----------
    collection: A pymongo.Collection instance.
    
    Returns
    -------
    An int. The number of documents modified.
    '''
    
    # YOUR CODE HERE
    count = len([item for item in collection.find({'conds':'Clear'})])
    collection.update_many({'conds':'Clear'}, {'$set':{'conds': 'Cloudy'}})
    
    return count

In [15]:
n_modified = modify_data(client[dbname]['0101'])
print('{0} records modified.'.format(n_modified))

6 records modified.


In [16]:
assert_equal(
    n_modified,
    len([obs['_id'] for obs in observations if obs['conds'] == 'Clear'])
    )

conds = [obs['conds'] for obs in observations]
for cond in conds:
    if cond != 'Clear' and cond != 'Cloudy':
        r = retrieve_data(client[dbname]['0101'], cond)
        n = [obs['_id'] for obs in observations if obs['conds'] == cond]
        assert_equal(len(r), len(n))
        assert_equal(set(r), set(n))

## Advanced Querying

- Find all documents with `visi` equal to `"10.0"` and sort the documents by `conds`.
- Return a list of `conds` as a validation of the process.

In [23]:
def query(collection):
    '''
    Finds all documents with "visi" equal to `"10.0"
    and sort the documents by "conds".
    
    Parameters
    ----------
    collection: A pymongo.Collection instance.

    Returns
    -------
    A list of strings.
    '''
    
    # YOUR CODE HERE
    result = [item['conds'] for item in collection.find({"visi": {'$eq': '10.0'}}).sort('conds')]

    return result

In [24]:
query_conds = query(client[dbname]['0101'])
print(query_conds)

['Cloudy', 'Cloudy', 'Cloudy', 'Cloudy', 'Cloudy', 'Cloudy', 'Mostly Cloudy', 'Mostly Cloudy', 'Mostly Cloudy', 'Mostly Cloudy', 'Overcast', 'Overcast', 'Partly Cloudy', 'Partly Cloudy', 'Partly Cloudy', 'Partly Cloudy', 'Partly Cloudy', 'Partly Cloudy', 'Scattered Clouds', 'Scattered Clouds', 'Scattered Clouds']


In [25]:
modified_conds = [obs['conds'] for obs in observations if obs['visi'] == '10.0']
modified_conds = ['Cloudy' if cond == 'Clear' else cond for cond in modified_conds]
modified_conds = sorted(modified_conds)
assert_equal(query_conds, modified_conds)

## Deleting Data

- Delete all documents whose `conds` attribute is equal to `"Cloudy"` from our collection.
- Return the number of documents deleted as a validation of the process.

In [26]:
def delete_data(collection):
    '''
    Deletes all documents whose "conds" == "Cloudy".
    
    Paramters
    ---------
    collection: A pymongo.Collection instance.

    Returns
    -------
    An int. The number of documents deleted.
    '''
    
    # YOUR CODE HERE
    count = len([item for item in collection.find({'conds':'Cloudy'})])
    collection.delete_many({'conds': 'Cloudy'})
    
    return count

In [27]:
n_deleted = delete_data(client[dbname]['0101'])
print('{0} records deleted.'.format(n_deleted))

6 records deleted.


In [28]:
deleted_obs = [obs for obs in modified_conds if obs == 'Cloudy']
assert_equal(n_deleted, len(deleted_obs))

for cond in set(conds):
    if cond != 'Clear' and cond != 'Cloudy':
        r = retrieve_data(client[dbname]['0101'], cond)
        n = [obs['_id'] for obs in observations if obs['conds'] == cond]
        assert_equal(len(r), len(n))
        assert_equal(set(r), set(n))

## Cleanup

When you are done or if you want to start over with a clean database, run the following code cell.

In [29]:
if dbname in client.database_names():
    client.drop_database(dbname)
    
assert_true(dbname not in client.database_names())